In [1]:
import os
import openai
import time
from tqdm import tqdm
from sklearn.metrics import f1_score, accuracy_score, classification_report

openai.api_key = os.getenv("OPENAI_API_KEY")

import json
import numpy as np

np.random.seed(0)
features_valid = json.load(open("../data/valid_subset_text.json"))
features_valid_erp = json.load(open("../dataset_bias/valid_subset_text_erp.json"))
features_valid_tense_all = json.load(open("../dataset_bias/valid_subset_text_tense_bias_vague.json"))

# features_valid_tense_vague = [item for item in features_valid_tense_all if item['labels'][0] == 3]
# features_valid_dep = json.load(open("../dataset_bias/valid_text_features_matres_dep_bias.json"))

# 1. zero_shot

"Determine the temporal order from \"{e1}\" to \"{e2}\" in the following sentence: \"{context}\". Only answer one word from AFTER, BEFORE, EQUAL, VAGUE. Answer:"

In [18]:
convert_dict = {'BEFORE':0, 'AFTER':1, 'EQUAL':2, 'VAGUE':3}
def parse_result(ans):
    return convert_dict.get(ans.upper(), 3)


In [8]:
zs_results = []
for i in tqdm(range(len(features_valid))):
    item = features_valid[i]
    context = item['text'].replace("[CLS]", "").replace("[SEP]", "").strip()
    e1 = item['e1']
    e2 = item['e2']
    
    prompt = f"Given the document {context} and a list of temporal relations [before, after, vague, equal] and event triggers {e1} and {e2}. what is the temporal relation between {e1} and {e2}? Answer vague if unsure. Keep the answer short and concise"
#     print(prompt)
    while True:
        try:
            zs_results.append(openai.Completion.create(
                        model="text-davinci-003",
                        prompt=prompt,
                        max_tokens=20,
                        temperature=0
            ))
            break
        except:
            time.sleep(10)
    time.sleep(2)
#     break

100%|███████████████████████████████████████████████████████████████████████████████████████████████████████████████| 1000/1000 [1:04:42<00:00,  3.88s/it]


In [ ]:
zs_results[0]

In [23]:
Counter([item['choices'][0]['text'].strip().replace(".", "").replace("\n", "") for item in zs_results])

Counter({'Vague': 935,
         '': 32,
         'After': 19,
         'Equal': 4,
         ' Vague': 1,
         'Before': 7,
         'equal': 1,
         'vague': 1})

In [20]:
ze_preds = [parse_result(item['choices'][0]['text'].strip().replace(".", "").replace("\n", "")) for item in zs_results]

In [21]:
from collections import Counter
Counter(ze_preds)

Counter({3: 969, 1: 19, 2: 5, 0: 7})

In [31]:
from collections import Counter
Counter([item['choices'][0]['text'].strip() for item in zs_results])

Counter({'BEFORE': 998, 'EQUAL': 2})

In [22]:
labels = [features_valid[i]['labels'][0] for i in range(len(features_valid))]

print(f1_score(labels, ze_preds, average='macro'), f1_score(labels, ze_preds, average="micro"))

0.05891182029303557 0.11200000000000002


# new

In [27]:
zs_results_new = []
for i in tqdm(range(len(features_valid))):
# for i in tqdm(range(100)):
    item = features_valid[i]
    context = item['text'].replace("[CLS]", "").replace("[SEP]", "").strip()
    e1 = item['e1']
    e2 = item['e2']
    
    prompt = f"Given the document {context} and a list of temporal relations [before, after, vague, equal] and event triggers {e1} and {e2}. what is the temporal relation between {e1} and {e2}? Select from the temporal relation list. Keep the answer short and concise"
#     print(prompt)
    while True:
        try:
            zs_results_new.append(openai.Completion.create(
                        model="text-davinci-003",
                        prompt=prompt,
                        max_tokens=20,
                        temperature=0
            ))
            break
        except:
            time.sleep(10)
    time.sleep(2)
#     break

100%|█████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 1000/1000 [52:34<00:00,  3.15s/it]


In [28]:
Counter([item['choices'][0]['text'].strip().replace(".", "").replace("\n", "") for item in zs_results_new])

Counter({'Equal': 569, 'Vague': 115, 'After': 261, 'Before': 55})

In [29]:
labels = [features_valid[i]['labels'][0] for i in range(len(features_valid))]
ze_preds_new = [parse_result(item['choices'][0]['text'].strip().replace(".", "").replace("\n", "")) for item in zs_results_new]

print(f1_score(labels, ze_preds_new, average='macro'), f1_score(labels, ze_preds_new, average="micro"))


0.1302917258119143 0.134


In [31]:
with open("prompts/prompt_3_zeroshot_new.json", "w") as writer:
    json.dump(ze_preds_new, writer)